<a href="https://colab.research.google.com/github/mohamad-hk/Influencing-factors-in-America-accidents/blob/main/Accident.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sobhanmoosavi/us-accidents
!unzip us-accidents.zip

Dataset URL: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
License(s): CC-BY-NC-SA-4.0
 97% 631M/653M [00:01<00:00, 273MB/s]
100% 653M/653M [00:01<00:00, 378MB/s]
Archive:  us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [2]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 603, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 603 (delta 131), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (603/603), 199.38 KiB | 16.61 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Installing RAPIDS remaining 25.08 libraries
Using Python 3.12.11 environment at: /usr
Resolved 177 packages in 11.14s
Prepared 54 packages in 59.06s
Uninstalled 32 packages in 1.02s
Installed 54 packages in 364ms
 + arrow==1.3.0
 - bokeh==3.7.3
 + bokeh==3.6.3
 + cucim-cu12==25.8.0
 + cuda-bindings==12.9.2
 + cuda-pathfinder==1.2.1
 - cuda-python==12.6.2.post1
 + cuda-python==12.9.2
 - cudf-cu12==25.6.0 (from https://pypi.nvidia.com/cudf-cu12/cudf_cu12-25.6.0-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl)
 + cudf-cu12==25.8.0
 + cugraph-cu12==25.8.0
 - cuml-cu12==25.6.0
 + cuml-cu12==25.8.0
 - cuvs-cu12==25.6.1
 + cuvs-

In [3]:
import numpy as np
import pandas as pd
import time
from tabulate import tabulate
import cudf
from sklearn.preprocessing import LabelEncoder

In [8]:
df=cudf.read_csv("US_Accidents_March23.csv")

In [5]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Source                 object
 2   Severity               int64
 3   Start_Time             object
 4   End_Time               object
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object
 11  Street                 object
 12  City                   object
 13  County                 object
 14  State                  object
 15  Zipcode                object
 16  Country                object
 17  Timezone               object
 18  Airport_Code           object
 19  Weather_Timestamp      object
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)            float64
 23  

In [9]:
df.drop(columns=["ID", "Description", "County"], inplace=True)
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 43 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 object
 1   Severity               int64
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float64
 5   Start_Lng              float64
 6   End_Lat                float64
 7   End_Lng                float64
 8   Distance(mi)           float64
 9   Street                 object
 10  City                   object
 11  State                  object
 12  Zipcode                object
 13  Country                object
 14  Timezone               object
 15  Airport_Code           object
 16  Weather_Timestamp      object
 17  Temperature(F)         float64
 18  Wind_Chill(F)          float64
 19  Humidity(%)            float64
 20  Pressure(in)           float64
 21  Visibility(mi)         float64
 22  Wind_Direction         object
 23

In [10]:
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df


converted_df=auto_data_type(df)

In [11]:
converted_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 43 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 object
 1   Severity               uint8
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float32
 5   Start_Lng              float32
 6   End_Lat                float32
 7   End_Lng                float32
 8   Distance(mi)           float32
 9   Street                 object
 10  City                   object
 11  State                  object
 12  Zipcode                object
 13  Country                object
 14  Timezone               object
 15  Airport_Code           object
 16  Weather_Timestamp      object
 17  Temperature(F)         float32
 18  Wind_Chill(F)          float32
 19  Humidity(%)            float32
 20  Pressure(in)           float32
 21  Visibility(mi)         float32
 22  Wind_Direction         object
 23

In [12]:
text_columns = [
    "Source", "City", "State", "Weather_Condition","Street",
    "Wind_Direction", "Sunrise_Sunset", "Civil_Twilight",
    "Nautical_Twilight", "Astronomical_Twilight"
]

for c in text_columns:
    encoder = LabelEncoder()
    encoded = encoder.fit_transform(converted_df[c].to_pandas().astype(str))
    converted_df[c] = cudf.Series(encoded).astype("uint16")


In [13]:
converted_df["Timezone"] = cudf.Series(encoder.fit_transform(converted_df["Timezone"].to_pandas())).astype("uint8")
converted_df["Airport_Code"] = cudf.Series(encoder.fit_transform(converted_df["Airport_Code"].to_pandas())).astype("uint8")

In [14]:
cols = ["Amenity", "Bump", "Crossing", "Give_Way", "Junction", "No_Exit",
        "Railway", "Roundabout", "Station", "Stop", "Traffic_Calming",
        "Traffic_Signal", "Turning_Loop"]

for col in cols:
    converted_df[col] = converted_df[col].astype("uint8")

In [15]:
converted_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 43 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 uint16
 1   Severity               uint8
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float32
 5   Start_Lng              float32
 6   End_Lat                float32
 7   End_Lng                float32
 8   Distance(mi)           float32
 9   Street                 uint16
 10  City                   uint16
 11  State                  uint16
 12  Zipcode                object
 13  Country                object
 14  Timezone               uint8
 15  Airport_Code           uint8
 16  Weather_Timestamp      object
 17  Temperature(F)         float32
 18  Wind_Chill(F)          float32
 19  Humidity(%)            float32
 20  Pressure(in)           float32
 21  Visibility(mi)         float32
 22  Wind_Direction         uint16
 23  